In [ ]:
"""
Created on Mon Oct 21 16:13 2024

Prepare the Justine file containing zmin and zmax

@author: Clara Burgard
"""

In [1]:
import xarray as xr
from tqdm.notebook import tqdm

In [2]:
%matplotlib qt5

QStandardPaths: error creating runtime directory '/run/user/2784' (Permission denied)


In [3]:
inputpath_raw = '/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/raw/'
inputpath_interim = '/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/interim/'

In [4]:
file_Justine = xr.open_dataset(inputpath_raw + 'Mask_Iceshelf_IMBIE2_v2.nc')
BedMachine_mask_2km= xr.open_dataset(inputpath_interim + 'BedMachine_v3_aggregated2km_masks_only.nc')
BedMachine_vars_2km= xr.open_dataset(inputpath_interim + 'BedMachine_v3_aggregated2km_allvars_withcategories.nc')


In [10]:
ice_draft = (BedMachine_vars_2km['thickness'] - BedMachine_vars_2km['surface']).sel(category=3).drop('category')

In [7]:
ice_draft.plot() # ice draft is automatically above 0 now

libGL error: No matching fbConfigs or visuals found
libGL error: failed to load driver: swrast


In [ ]:
file_Justine['Iceshelf'].plot(vmax=10)

In [ ]:
kisf = 8
ice_draft.where(file_Justine['Iceshelf'] == kisf).max()

In [ ]:
BedMachine_mask_2km['isf_conc']

In [19]:
 isf_mask = (file_Justine['Iceshelf_extrap'] == kisf) & (BedMachine_mask_2km['isf_conc'] > 0)

In [25]:
BedMachine_mask_2km['isf_conc'].where(file_Justine['Iceshelf_extrap'] == 3).plot()

In [18]:
(file_Justine['Iceshelf_extrap'] == kisf).plot()

In [50]:
kisf = 64
isf_mask = (file_Justine['Iceshelf_extrap'] == kisf) & (BedMachine_mask_2km['isf_conc'] > 0)

In [51]:
ice_draft.where(isf_mask).plot()

In [26]:
zmin_list = []
zmax_list = []
perc_01_list = []
perc_99_list = []
for kisf in tqdm(file_Justine['ID']):
    isf_mask = (file_Justine['Iceshelf_extrap'] == kisf) & (BedMachine_mask_2km['isf_conc'] > 0)
    
    zmin = ice_draft.where(isf_mask).min()
    zmin_list.append(zmin.assign_coords({'ID': kisf}))
    zmax = ice_draft.where(isf_mask).max()
    zmax_list.append(zmax.assign_coords({'ID': kisf}))
    perc_01 = ice_draft.where(isf_mask).quantile(0.01)
    perc_01_list.append(perc_01.assign_coords({'ID': kisf}))
    perc_99 = ice_draft.where(isf_mask).quantile(0.99)
    perc_99_list.append(perc_99.assign_coords({'ID': kisf}))

zmin_all = xr.concat(zmin_list, dim='ID')
zmax_all = xr.concat(zmax_list, dim='ID')
perc_01_all = xr.concat(perc_01_list, dim='ID')
perc_99_all = xr.concat(perc_99_list, dim='ID')

  0%|          | 0/133 [00:00<?, ?it/s]

/home/cburgard/.conda/envs/py38/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1556: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,
/home/cburgard/.conda/envs/py38/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1556: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,
/home/cburgard/.conda/envs/py38/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1556: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,
/home/cburgard/.conda/envs/py38/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1556: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,
/home/cburgard/.conda/envs/py38/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1556: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,
/home/cburgard/.conda/envs/py38/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1556: RuntimeWarning: All-NaN slice encountered
  return function_bas

In [27]:
file_Justine['z_min'] = zmin_all
file_Justine['z_min'].attrs['standard_name'] = 'most shallow point of the ice-shelf draft'
file_Justine['z_min'].attrs['units'] = 'm below sea level'

file_Justine['z_max'] = zmax_all
file_Justine['z_max'].attrs['standard_name'] = 'deepest point of the ice-shelf draft'
file_Justine['z_max'].attrs['units'] = 'm below sea level'

file_Justine['z_perc01'] = perc_01_all
file_Justine['z_perc01'].attrs['standard_name'] = '1st percentile over ice-shelf draft'
file_Justine['z_perc01'].attrs['units'] = 'm below sea level'

file_Justine['z_perc99'] = perc_99_all
file_Justine['z_perc99'].attrs['standard_name'] = '99th percentile over ice-shelf draft'
file_Justine['z_perc99'].attrs['units'] = 'm below sea level'

file_Justine['isf_conc'] = BedMachine_mask_2km['isf_conc']
file_Justine['isf_conc'].attrs['standard_name'] = 'Ice-shelf concentration in the grid cell between 0 and 1'

In [54]:
file_Justine.to_netcdf(inputpath_interim + 'Mask_Iceshelf_IMBIE2_v2_with_zmin_zmax_isfconc.nc')

In [52]:
for iid in file_Justine.ID:
    print(iid.values,file_Justine['NAME'].sel(ID=iid).values,file_Justine['z_perc01'].sel(ID=iid).values)

1 Jelbart 169.54441069722176
2 Fimbul 81.8926885253191
3  nan
4 Vigrid 150.67135648727418
5 Nivl 92.9348541995883
6  nan
7 Lazarev 67.20663256406783
8 Borchgrevink 47.26678985916078
9 Baudouin 81.90099007710815
10  nan
11 Prince_Harald 31.0103200571239
12  nan
13 Shirase 88.28886233061552
14 Hannan/Telen/Skallen 79.19220874473453
15 Rayner_Thyer 143.3075850406289
16 Zubchatyy/Porter/Myers 33.200275969319044
17 WilmaRobertDowner 116.57360565379263
18 Edward_VIII 47.69272315144539
19  nan
20 Utsikkar/Mulebreen/Cirque_Fjord/Hoseason/Rund_Bay 134.50996131271123
21 Amery 158.85066553890704
22 Publications 66.63684043377638
23 Sorsdal/Hovde/Flatnes/Dalk 377.37461585760116
24 West 46.42410892903805
25 Helen 17.38851813916117
26 Shackleton 52.8071139908582
27 Tracy_Tremenchus 17.50194251704961
28  nan
29 Conger_Glenzer 39.5865545091778
30  nan
31 Vincennes_Bay/Underwood 198.3181858357787
32 Williamson/Whittle/FoxE 29.83499250166118
33 Totten 166.97531796261669
34 Moscow_University 180.06558987

In [ ]:
kisf = 2
ice_draft.where((file_Justine['Iceshelf_extrap'] == kisf) & (BedMachine_mask_2km['isf_conc'] > 0) & (ice_draft == 0)).plot()

In [ ]:
BedMachine_mask_2km['ocean_conc'].where((file_Justine['Iceshelf_extrap'] == kisf) & (ice_draft == 0) & (BedMachine_mask_2km['isf_conc'] > 0), drop=True).plot()

In [ ]:
((ice_draft == 0) & (BedMachine_mask_2km['isf_conc'] > 0)).plot()